In [1]:
import os
import numpy as np
import glob
import pandas as pd  
import random
import math as mp
import pickle
#import matplotlib.pyplot as plt
import tensorflow as tf
import time
from pickle import load

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

%matplotlib inline
#import matplotlib.pyplot as plt

from tensorflow import keras
import tensorflow.keras.backend as K


from numpy import array
from numpy import hstack
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.compat.v1.keras.layers import CuDNNLSTM, CuDNNGRU
from tensorflow.keras.layers import Dense,Flatten,Dropout, MaxPooling1D

from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import GlobalMaxPooling1D
from keras.callbacks import ModelCheckpoint, EarlyStopping



from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from pyquaternion import Quaternion

import toml

from math import floor

from ipynb.fs.full.V06_Utils  import *
from ipynb.fs.full.V06_Prepare_dataset import *

In [2]:
######################################################################
# Init Configuration
######################################################################

config_path = 'C:/Users/weikert1/Documents/Thesis/Project_V06_26072021/Code/V06_config.toml'
cfg = toml.load(config_path)

# General 
n_in_seq = cfg['n_in_seq_quat']
n_out_seq = cfg['n_out_seq_quat']
n_steps = cfg['n_steps'] 
LAT_in_rows = cfg['LAT_in_rows'] #6*16,66ms= 64ms

# model ID --> specify this for every new model
model_id = cfg['1_2_model_ID']

# Directories 
Path_dataset01_test = cfg['Path_dataset01']
Path_results_dataset01_LSTM_quat_model_id = cfg['Path_results_dataset01'] + cfg['1_model_type'] + "/" + model_id  + "/" 

# Tensorflow configuration
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)
print("Num GPUs Available: ", len(tf.config.list_physical_devices("GPU")))

Num GPUs Available:  1


In [3]:
######################################################################
# Load Training Data
######################################################################

X_train, X_val, y_train, y_val = prepare_training_data_quat(Path_dataset01_test)

print("X train shape is: " + str(X_train.shape))
print("X val shape is: " + str(X_val.shape))
      
print("y train shape is: " + str(y_train.shape))
print("y val shape is: " + str(y_val.shape))

X shape is: (71405, 15, 4)
y shape is: (71405, 4)
------
X train shape is: (57124, 15, 4)
X val shape is: (14281, 15, 4)
y train shape is: (57124, 4)
y val shape is: (14281, 4)


In [4]:
print(y_train[0])

[ 0.7340308   0.02212691  0.6768867  -0.05033436]


In [5]:
def conjugate(q):
    mult = tf.constant(np.array([1,-1,-1,-1])[np.newaxis], dtype=np.float32)
    return q*mult

def inverse(q):
    return conjugate(q) / tf.reduce_sum(q*q, axis=-1, keepdims=True)

def log(q):
    v = q[:, 1:]
    a = q[:, :1]
    q_norm = tf.norm(q, axis=-1, keepdims=True)
    x = a / q_norm
    eps = np.finfo(np.float32).eps * 8.0
    x *= (1.0 - eps)
    vec_part = tf.nn.l2_normalize(v, axis=-1) * tf.math.acos(x)
    real_part = tf.math.log(q_norm)
    return tf.concat([real_part, vec_part], axis=-1)

def mult(quaternion1, quaternion2):
    w1, x1, y1, z1 = tf.unstack(quaternion1, axis=-1)
    w2, x2, y2, z2 = tf.unstack(quaternion2, axis=-1)
    x = x1 * w2 + y1 * z2 - z1 * y2 + w1 * x2
    y = -x1 * z2 + y1 * w2 + z1 * x2 + w1 * y2
    z = x1 * y2 - y1 * x2 + z1 * w2 + w1 * z2
    w = -x1 * x2 - y1 * y2 - z1 * z2 + w1 * w2
    return tf.stack((w, x, y, z), axis=-1)
    
def geodesic_dist(q1, q2):
    q1 = q1/tf.norm(q1)
    q2 = q2/tf.norm(q2)
    x = mult(inverse(q1), q2)
    x = tf.norm(log(x), ord = 'euclidean', axis=-1)
    return tf.reduce_mean(x)


def angle_dist(q1, q2):
    x = tf.reduce_sum(q1*q2, axis=-1)
    eps = np.finfo(np.float32).eps * 8.0
    x *= (1.0 - eps)
    x = 2*tf.math.acos(x)
    return tf.reduce_mean(x)


def quat_antipodal_loss(y_true, y_pred):
    dist1 = tf.reduce_mean(tf.abs(y_true-y_pred), axis=-1)
    dist2 = tf.reduce_mean(tf.abs(y_true+y_pred), axis=-1)
    loss = tf.where(dist1<dist2, dist1, dist2)
    return tf.reduce_mean(loss)


def euler_angles_loss(y_true, y_pred):
    dist1 = tf.abs(y_true - y_pred)
    dist2 = tf.abs(2*np.pi + y_true - y_pred)
    dist3 = tf.abs(-2*np.pi + y_true - y_pred)
    loss = tf.where(dist1<dist2, dist1, dist2)
    loss = tf.where(loss<dist3, loss, dist3)
    return tf.reduce_mean(loss)


def mean_angle_btw_vectors(v1, v2):
    dot_product = tf.reduce_sum(v1*v2, axis=-1)
    cos_a = dot_product / (tf.norm(v1, axis=-1) * tf.norm(v2, axis=-1))
    eps = 1e-8
    cos_a = tf.clip_by_value(cos_a, -1 + eps, 1 - eps)
    angle_dist = tf.math.acos(cos_a) / np.pi * 180.0
    return tf.reduce_mean(angle_dist)

def minmax_inverse_transform(y_true, y_pred):
    path_scaler_objects = cfg['Path_scaler_object_quat']
    w_min, x_min, y_min, z_min, w_max, x_max, y_max, z_max = load_scaler_objects(path_scaler_objects)
    
    w1, x1, y1, z1 = tf.unstack(y_true, axis=-1)
    w2, x2, y2, z2 = tf.unstack(y_pred, axis=-1)
    
    w1, w2 = (w1 - K.constant(w_min)) / K.constant(w_max - w_min) , (w2 - K.constant(w_min)) / K.constant(w_max - w_min)
    x1, x2 = (x1 - K.constant(x_min)) / K.constant(x_max - x_min) , (x2 - K.constant(x_min)) / K.constant(x_max - x_min)
    y1, y2 = (y1 - K.constant(y_min)) / K.constant(y_max - y_min) , (y2 - K.constant(y_min)) / K.constant(y_max - y_min)
    z1, z2 = (z1 - K.constant(z_min)) / K.constant(z_max - z_min) , (z2 - K.constant(z_min)) / K.constant(z_max - z_min)

    y_true = tf.stack((w1, x1, y1, z1), axis=-1)
    y_pred = tf.stack((w2, x2, y2, z2), axis=-1)
    return y_true, y_pred

def ang_mae(y_true,y_pred):
    y_true, y_pred = minmax_inverse_transform(y_true, y_pred)
    ang_dist_rad = geodesic_dist(y_true, y_pred) #returns a value between 0 and pi
    #ang_dist_deg = rad2deg(ang_dist_rad)
    shape = tf.cast(tf.size(ang_dist_rad),dtype=np.float32)
    ang_mae =  K.sum(ang_dist_rad)/shape
    return rad2deg(tf.reduce_mean(ang_mae))


def rad2deg(rad):
    pi_on_180 = 0.017453292519943295
    return rad / pi_on_180


def absolute_distance(q0, q1):
    q0_minus_q1 = q0 - q1
    q0_plus_q1  = q0 + q1
    d_minus =  tf.norm(q0_minus_q1, axis=-1, keepdims=True) 
    d_plus  = tf.norm(q0_plus_q1, axis=-1, keepdims=True) 
    check = tf.math.less(d_minus, d_plus)
    result = tf.where(check == True , d_minus, d_plus)
    return tf.reduce_mean(result)


def rotation_loss(q1, q2):
    q1 = q1/tf.norm(q1)
    q2 = q2/tf.norm(q2)
    loss = q1 - q2
    loss = tf.norm(loss)
    return rad2deg(loss)


def rotation_loss_2(y_true,y_pred):
    rotation_loss = tf.sqrt(tf.nn.l2_loss(y_true-y_pred))
    return tf.reduce_mean(rotation_loss)


In [6]:
def My_LSTM(X_train, y_train, Neurons, Dropout_rate,Learning_rate):

    # the dataset knows the number of features, e.g. 2
    n_steps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]


    
    ...
    # define model
    #model = Sequential()
    
    #model.add(CuDNNLSTM(100, return_sequences=True, input_shape=(n_steps, n_features)))
    #model.add(Dropout(0.4))
    
    #model.add(CuDNNLSTM(100))
    #model.add(Dropout(0.4))
    
    #model.add(CuDNNLSTM(100))
    #model.add(Dropout(0.4))
     
    #model.add(Dense(100, activation='sigmoid'))
    #model.add(Dense(n_features, activation='sigmoid'))
    
    model = Sequential()
    model.add(CuDNNLSTM(Neurons, input_shape=(n_steps, n_features),return_sequences=True))
    model.add(Dropout(Dropout_rate))
    #model.add(CuDNNLSTM(128))
    #model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_features))
    
    
    opt = tf.keras.optimizers.SGD(learning_rate=Learning_rate)
    #model.compile(opt, loss=tf.keras.losses.Huber() , metrics = geodesic_dist) #run_eagerly=True
    model.compile(opt, loss=angle_dist) #run_eagerly=True

    model.summary()
    
    
    
    
    
    return model


In [7]:
#################################################
print("-----")
#model = My_LSTM(X_train, y_train)

-----


# Train the mdel 

In [8]:
# init model checkpoint
#mc = ModelCheckpoint(Path_results_dataset01_LSTM_quat_model_id + "/Model/" + model_id + '_CUDNNLSTM_quat_best_model.h5', monitor ='val_loss', mode='min', save_best_only=True, verbose=1)
#es = EarlyStopping(monitor='val_loss', mode='min', patience=30)

In [9]:
#quat_hist = model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=Batch_size, epochs=Epochs, verbose=1, callbacks=[mc] )

In [10]:
def save_model(model, path):
# serialize model to YAML
    model_yaml = model.to_yaml()
    with open(path +  "_CUDNN_arch.yaml", "w") as yaml_file:
        yaml_file.write(model_yaml)


    #model.save_weights(save_path + "V05_CUDNNLSTM_rotation_20072021.h5")
    return print("Saved model to disk")

In [11]:
#LAT_in_rows = [1,2,3,4,5,6,7,8,9,10]
Neurons = [64,128,256,512]
Dropout_rate = [10,20,30,40,50,60,70,80,90]
Learning_rate = [10,100,1000,10000,100000,1000000,10000000]
Batch_size = [1028,512,256,128,64]
Epochs = [100,200,300,400,500]
#Loss = [quat_antipodal_loss,absolute_distance,geodesic_dist]

In [12]:
from keras.callbacks import CSVLogger

In [14]:
for Neurons_i in Neurons:
    for Dropout_rate_i in Dropout_rate:
        for Learning_rate_i in Learning_rate:
            for Batch_size_i in Batch_size:
                for Epochs_i in Epochs:
                        X_train, X_val, y_train, y_val = prepare_training_data_quat(Path_dataset01_test)
                        model = My_LSTM(X_train, y_train, Neurons_i, 1/Dropout_rate_i , 5/Learning_rate_i)
                        individual_name = "Neurons_" + str(Neurons_i) + "_" + "_DR_" + str(Dropout_rate_i) + "__" + "LR_" + str(Learning_rate_i) + "__" + "Batch_" + str(Batch_size_i) + "__" + "Epochs_" + str(Epochs_i) 
                        mc = ModelCheckpoint(Path_results_dataset01_LSTM_quat_model_id + "/Model/" + individual_name +  '_CUDNN_best_model.h5', monitor ='val_loss', mode='min', save_best_only=True, verbose=0)
                        csv_logger = CSVLogger(Path_results_dataset01_LSTM_quat_model_id + "/Model/" + individual_name + '.csv', append=True, separator=';')
                        history = model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=Batch_size_i, epochs=Epochs_i, verbose=1, callbacks=[mc,csv_logger] )
                        loss_history = history.history["loss"]
                        plt.plot(history.history['loss'])
                        plt.plot(history.history['val_loss'])
                        plt.title('model loss')
                        plt.ylabel('loss')
                        plt.xlabel('epoch')
                        plt.legend(['train', 'test'], loc='upper left')
                        plt.show()
                        save_model(model,Path_results_dataset01_LSTM_quat_model_id + "/Model/" + individual_name )
                        tf.keras.backend.clear_session()






                            

X shape is: (71405, 15, 4)
y shape is: (71405, 4)
------
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm (CuDNNLSTM)       (None, 15, 64)            17920     
_________________________________________________________________
dropout (Dropout)            (None, 15, 64)            0         
_________________________________________________________________
flatten (Flatten)            (None, 960)               0         
_________________________________________________________________
dense (Dense)                (None, 100)               96100     
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 404       
Total params: 114,424
Trainable params: 114,424
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100


InternalError:  Blas xGEMM launch failed : a.shape=[1,1028,960], b.shape=[1,960,100], m=1028, n=100, k=960
	 [[node sequential/dense/MatMul (defined at <ipython-input-14-08bdaa99e1c2>:11) ]] [Op:__inference_train_function_1359]

Function call stack:
train_function
